In [23]:
using ExpFamilyPCA

n_samples = 100
n_dims = 30
X = rand(n_samples, n_dims)  # generate fake probability data
X ./= sum(X, dims=2)

n_components = 2
epca = PoissonPCA(n_components, n_dims)
fit!(epca, X; verbose=true, maxiter=50)

X̃1 = compress(epca, X)
recon1 = decompress(epca, X̃1)

3723.604496511967


3119.252242659718


2732.170294468625


2453.9359384726195


2334.2379924369325


...

1668.7162541417997


1664.9901904542648


1661.1373948802773


1657.1914136066762


100×30 Matrix{Float64}:
 0.00267027   0.880563  0.880619  …  0.00312499   0.88065   0.00258198
 0.0236374    0.931033  0.931069     0.0271028    0.931085  0.0237297
 0.0104136    0.92137   0.921414     0.0125749    0.921429  0.0106179
 0.00270431   0.894836  0.894891     0.00338058   0.894914  0.00273419
 0.000495997  0.860712  0.860782     0.000641881  0.860813  0.000493225
 0.0218376    0.933748  0.933785  …  0.0255818    0.933798  0.0221997
 0.0141447    0.918299  0.918341     0.0162525    0.91836   0.0140494
 0.000179691  0.860377  0.860455     0.000261327  0.860479  0.000188705
 0.00737848   0.915876  0.915923     0.00904524   0.915939  0.00753854
 0.00290354   0.897157  0.897212     0.003639     0.897233  0.00294563
 ⋮                                ⋱                         
 0.010619     0.919618  0.919662     0.0126921    0.919678  0.0107579
 0.000580401  0.86293   0.862998     0.000745479  0.863029  0.000576371
 0.0244882    0.93512   0.935157     0.0284812    0.935169  0.024

In [21]:
using MultivariateStats

M = fit(PCA, X; maxoutdim=n_components)
X̃2 = predict(M, X)
recon2 = reconstruct(M, X̃2)

20×30 Matrix{Float64}:
 4.66405  3.28828   6.14938  4.86417  …  5.52828  5.06016  4.78345  3.66033
 7.22794  7.67265   8.6317   5.56508     4.81418  5.1751   3.71935  7.11116
 4.17289  2.74542   4.86172  5.10359     6.03527  5.45473  5.86671  3.33108
 6.45575  7.69715   6.15615  5.39234     4.49687  5.03505  4.43117  7.1177
 5.16959  4.92751   6.42925  4.3579      4.1906   4.2193   3.35178  4.75916
 6.64637  8.74855   5.80741  5.12692  …  3.70494  4.57939  3.82879  7.84498
 4.95375  5.96327   4.8271   3.98972     3.22821  3.67888  3.09184  5.46466
 4.29828  3.24354   2.95944  6.55352     7.76789  7.13335  8.95335  4.10904
 5.23289  3.50561   7.86475  4.83347     5.44895  4.94656  3.97003  3.79308
 6.14179  4.79822  10.3917   3.96248     3.81425  3.66854  1.10226  4.50594
 5.51206  6.92703   3.90421  5.37425  …  4.71403  5.18657  5.53693  6.56316
 4.75759  3.90115   4.64312  5.76369     6.47289  6.06587  6.74591  4.37613
 5.79188  7.10886   4.38468  5.58787     4.94797  5.39771  5.65058

In [22]:
using LinearAlgebra
using Statistics

dist1 = X .- recon1
dist2 = X .- recon2

KL_divergence(p, q) = sum(p .* log.(p ./ q))
KL1 = mean([KL_divergence(p, q) for (p, q) in zip(eachrow(X), eachrow(recon1))])
KL2 = mean([KL_divergence(p, normalize(abs.(q), 1)) for (p, q) in zip(eachrow(X), eachrow(recon2))])

# From theory, we expect the EPCA reconstruction to do better on the L1 norm but worse on L2
println("EPCA L2:", norm(dist1, 2))
println("PCA L2:", norm(dist2, 2))
println()
println("EPCA KL:", KL1)
println("PCA KL:", KL2)

EPCA L2:70.43625010270905
PCA L2:59.609249527403364

EPCA KL:34.93761923913772
PCA KL:828.9722271856319
